# Load Util File with TARE Model Functions

In [1]:
import os

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

# import from cmu-tare-model package
from config import PROJECT_ROOT
import pandas as pd

# Set columns in display
pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_columns') # Reset options to default

# Set rows in display
# pd.set_option('display.max_rows', None)
# pd.reset_option('display.max_rows') # Reset options to default

# import seaborn library (wrapper of matplotlib)
import seaborn as sns
sns.set_theme(style="darkgrid")

from datetime import datetime

# Get the current datetime
# Start the timer
start_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Storing Result Outputs in output_results folder
relative_path = os.path.join("cmu_tare_model", "output_results")
output_folder_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"""
-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------
      
Project root directory: {PROJECT_ROOT}
Result outputs will be exported here: {output_folder_path}

""")

Project root directory: c:\users\14128\research\cmu-tare-model

-------------------------------------------------------------------------------------------------------
Welcome to the Trade-off Analysis of residential Retrofits for energy Equity Tool (TARE Model)
Let's start by reading the data from the NREL EUSS Database.

Make sure that the zipped folders stay organized as they are once unzipped.
If changes are made to the file path, then the program will not run properly.
-------------------------------------------------------------------------------------------------------
      
Project root directory: c:\users\14128\research\cmu-tare-model
Result outputs will be exported here: c:\users\14128\research\cmu-tare-model\cmu_tare_model\output_results




In [2]:
# df_euss_am_baseline['county_fips'] = df_euss_am_baseline['in.county']
# df_euss_am_baseline['fips'] = df_euss_am_baseline['in.county'].apply(lambda x: x[1:3] + x[4:7])
# df_euss_am_baseline

# # Drop duplicate rows so that only the unique county FIPS codes remain.
# df_unique_counties = df_euss_am_baseline[['fips']].drop_duplicates()

# # Sort the DataFrame in ascending order (small to large)
# df_unique_counties = df_unique_counties.sort_values(by='fips')

# # Optional: Reset the index if desired
# df_unique_counties.reset_index(drop=True, inplace=True)

# # Write the sorted unique county FIPS codes to CSV
# df_unique_counties.to_csv("rcm_msc_CACES_counties.csv", index=False)
# df_unique_counties

# Simulate Residential Energy Consumption using NREL End-Use Savings Shapes

In [3]:
from cmu_tare_model.energy_consumption_and_metadata.load_and_filter_euss_data_v2 import *

# Measure Package 0: Baseline
menu_mp = 0
input_mp = 'baseline'

filename = "baseline_metadata_and_annual_results.csv"
relative_path = os.path.join("cmu_tare_model", "data", "euss_data", "resstock_amy2018_release_1.1", "state", filename)

file_path = os.path.join(PROJECT_ROOT, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Fix DtypeWarning error in columns:
# 'in.neighbors', 'in.geometry_stories_low_rise', 'in.iso_rto_region', 'in.pv_orientation', 'in.pv_system_size'
columns_to_string = {11: str, 61: str, 121: str, 103: str, 128: str, 129: str}
df_euss_am_baseline = pd.read_csv(file_path, dtype=columns_to_string, index_col="bldg_id") # UPDATE: Set index to 'bldg_id'

# Filter for occupied homes
occupancy_filter = df_euss_am_baseline['in.vacancy_status'] == 'Occupied'
df_euss_am_baseline = df_euss_am_baseline.loc[occupancy_filter]

# Filter for single family home building type
house_type_list = ['Single-Family Attached', 'Single-Family Detached']
house_type_filter = df_euss_am_baseline['in.geometry_building_type_recs'].isin(house_type_list)
df_euss_am_baseline = df_euss_am_baseline.loc[house_type_filter]

# Choose between national or sub-national level analysis
menu_state = get_menu_choice(menu_prompt, {'N', 'Y'})   # This code is only run in baseline

# National Level 
if menu_state == 'N':
    print("You chose to analyze all of the United States.")
    input_state = 'National'
    location_id = 'National'

# Filter down to state or city
else:
    input_state = get_state_choice(df_euss_am_baseline)    
    print(f"You chose to filter for: {input_state}")

    location_id = str(input_state)
    print(f"Location ID is: {location_id}")


    state_filter = df_euss_am_baseline['in.state'].eq(input_state)
    df_euss_am_baseline = df_euss_am_baseline.loc[state_filter]

    print(city_prompt)
    print(df_euss_am_baseline['in.city'].value_counts())

    menu_city = get_menu_choice(city_menu_prompt, {'N', 'Y'})

    # Filter for the entire selected state
    if menu_city == 'N':
        print(f"You chose to analyze all of state: {input_state}")
        
        location_id = str(input_state)
        print(f"Location ID is: {location_id}")
        
    # Filter to a city within the selected state
    else:
        input_cityFilter = get_city_choice(df_euss_am_baseline, input_state)
        print(f"You chose to filter for: {input_state}, {input_cityFilter}")

        location_id = input_cityFilter.replace(', ', '_').strip()
        print(f"Location ID is: {location_id}")

        city_filter = df_euss_am_baseline['in.city'].eq(f"{input_state}, {input_cityFilter}")
        df_euss_am_baseline = df_euss_am_baseline.loc[city_filter]

print(f"""
-------------------------------------------------------------------------------------------------------
BASELINE (Measure Package 0)
-------------------------------------------------------------------------------------------------------
DATAFRAME: df_euss_am_baseline

DATA: NREL EUSS Database
HOUSING FILTERS: Occupied units and Single Family Homes
GEOGRAPHIC FILTERS: National, State, or City

Additional details and documentation: 
      data can be found in the EUSS documentation here: 
methods can be found in the load_and_filter_euss_data.py file.
      
DATAFRAME: df_euss_am_baseline
      
{df_euss_am_baseline}
""")

Retrieved data for filename: baseline_metadata_and_annual_results.csv
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\euss_data\resstock_amy2018_release_1.1\state\baseline_metadata_and_annual_results.csv
You chose to analyze all of the United States.

-------------------------------------------------------------------------------------------------------
BASELINE (Measure Package 0)
-------------------------------------------------------------------------------------------------------
DATAFRAME: df_euss_am_baseline

DATA: NREL EUSS Database
HOUSING FILTERS: Occupied units and Single Family Homes
GEOGRAPHIC FILTERS: National, State, or City

Additional details and documentation: 
      data can be found in the EUSS documentation here: 
methods can be found in the load_and_filter_euss_data.py file.
      
DATAFRAME: df_euss_am_baseline
      
         upgrade      weight  applicability  in.sqft  \
bldg_id                                                
239 

In [4]:
# df_baseline_enduse(df_baseline, df_enduse, category, fuel_filter='Yes', tech_filter='Yes')
df_euss_am_baseline_home = df_enduse_refactored(df_baseline = df_euss_am_baseline,
                                                fuel_filter = 'Yes',
                                                tech_filter = 'Yes',
                                                invalid_row_handling='filter',
                                                )

print(f"""
In addition to the housing type and occupancy filters, the data has been filtered for fuel and technology:
      
FUEL FILTERS: 
      - Water Heating and Space Heating: Electricity, Fuel Oil, Natural Gas, Propane
      - Clothes Drying and Cooking: Electricity, Natural Gas, Propane

ACCEPTABLE TECH FILTERS (cost data available):
      - Water Heating: ['Electric Heat Pump, 80 gal', 'Electric Premium', 'Electric Standard', 
                        'Fuel Oil Premium', 'Fuel Oil Standard', 
                        'Natural Gas Premium', 'Natural Gas Standard', 
                        'Propane Premium', 'Propane Standard']
      - Space Heating: ['Electricity ASHP', 'Electricity Baseboard', 'Electricity Electric Boiler', 'Electricity Electric Furnace',
                        'Fuel Oil Fuel Boiler', 'Fuel Oil Fuel Furnace', 
                        'Natural Gas Fuel Boiler', 'Natural Gas Fuel Furnace', 
                        'Propane Fuel Boiler', 'Propane Fuel Furnace']
      - Clothes Drying and Cooking: Required no tech filter

df_enduse_refactored will display the number of representative homes after filtering for fuel and tech.
      
DATAFRAME: df_euss_am_baseline_home
      
{df_euss_am_baseline_home}
""")

Processing column: in.clothes_dryer
Initial data types: object
Data types after processing: object
Processing column: in.cooking_range
Initial data types: object
Data types after processing: object
331531 rows remain after applying total heating consumption calculation
Filtered for allowed fuels in heating: ['Natural Gas', 'Electricity', 'Propane', 'Fuel Oil']
321357 rows remain after applying heating fuel filter/mask
Filtered for allowed technologies in heating: ['Electricity ASHP', 'Electricity Baseboard', 'Electricity Electric Boiler', 'Electricity Electric Furnace', 'Fuel Oil Fuel Boiler', 'Fuel Oil Fuel Furnace', 'Natural Gas Fuel Boiler', 'Natural Gas Fuel Furnace', 'Propane Fuel Boiler', 'Propane Fuel Furnace']
291558 rows remain after applying heating technology filter/mask
291558 rows remain after applying total waterHeating consumption calculation
Filtered for allowed fuels in waterHeating: ['Natural Gas', 'Electricity', 'Propane', 'Fuel Oil']
291191 rows remain after applyin

## Project Future Energy Consumption Using EIA Heating Degree Day (HDD) Forecasted Data (Factors)

In [5]:
from cmu_tare_model.energy_consumption_and_metadata.project_future_energy_consumption import *

# Make a copy of the dataframe
df_baseline_scenario_consumption = df_euss_am_baseline_home.copy()

# Project Future Energy Consumption
df_euss_am_baseline_home, df_baseline_scenario_consumption = project_future_consumption(df=df_euss_am_baseline_home,
                                                                                        lookup_hdd_factor=lookup_hdd_factor,
                                                                                        menu_mp=menu_mp
                                                                                        )
# Display the baseline scenario summary dataframe
# df_euss_am_baseline_home
print(F"""
-------------------------------------------------------------------------------------------------------
PROJECT FUTURE ENERGY CONSUMPTION: Baseline Consumption
-------------------------------------------------------------------------------------------------------
Creating dataframe to store annual energy consumption calculations ...
      
DATAFRAME: df_euss_am_baseline_home

{df_euss_am_baseline_home}      

DATAFRAME: df_baseline_scenario_consumption
      
{df_baseline_scenario_consumption}
""")

Retrieved data for filename: aeo_projections_2022_2050.xlsx
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\projections\aeo_projections_2022_2050.xlsx
Projecting Future Energy Consumption (Baseline Equipment): heating
Projecting Future Energy Consumption (Baseline Equipment): waterHeating
Projecting Future Energy Consumption (Baseline Equipment): clothesDrying
Projecting Future Energy Consumption (Baseline Equipment): cooking

-------------------------------------------------------------------------------------------------------
PROJECT FUTURE ENERGY CONSUMPTION: Baseline Consumption
-------------------------------------------------------------------------------------------------------
Creating dataframe to store annual energy consumption calculations ...
      
DATAFRAME: df_euss_am_baseline_home

         square_footage census_region     census_division  \
bldg_id                                                     
239              1690.0         Sout

# Public Perspective: Monetized Marginal Damages from Emissions

## Fossil Fuels: Climate and Health-Related Pollutants

In [6]:
# fuel_types = ['electricity', 'naturalGas', 'propane', 'fuelOil']
# categories = ['heating', 'waterHeating', 'clothesDrying', 'cooking']

# df_copy = df_euss_am_baseline_home.copy()

# cols_to_check = [
#     'county_fips',
#     'base_heating_fuel',
#     # 'base_waterHeating_fuel',
#     # 'base_clothesDrying_fuel',
#     # 'base_cooking_fuel',
#     'base_electricity_heating_consumption',
#     'base_fuelOil_heating_consumption',
#     'base_naturalGas_heating_consumption',
#     'base_propane_heating_consumption',
#     'baseline_heating_consumption',
#     ]

# df_test = df_copy[cols_to_check]
# df_test

In [7]:
""""
We adjust all monetary values for inflation using the Bureau of Labor Statistics (BLS) 
Consumer Price Index (CPI) for All Urban Consumers (CPI-U).

inflation_adjustment.py does the following:
- Loads the BLS CPI for 2005-2023
- Uses this data to create a dataframe named df_bls_cpiu
- Calculates the CPI ratio constants for use later in the model (ex: cpi_ratio_2023_2020 for EPA SCC, etc.)

Additional information concerning the BLS CPI for All Urban Consumers (CPI-U) is provided the inflation_adjustment.py file.
"""

from cmu_tare_model.utils.inflation_adjustment import *
print(f"""
--------------------------------------------------------------------------------------------------------------------------------------
Inflate Marginal Social Cost (Damage) Factors using BLS CPI for All Urban Consumers (CPI-U)
--------------------------------------------------------------------------------------------------------------------------------------
DATAFRAME: Annual CPI-U for 2005-2023 used for cpi_ratio constants and inflation adjustments
      
{df_bls_cpiu}
""")

Retrieved data for filename: bls_cpiu_2005-2023.xlsx
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\inflation_data\bls_cpiu_2005-2023.xlsx
    year  cpiu_annual
0   2005      195.300
1   2006      201.600
2   2007      207.342
3   2008      215.303
4   2009      214.537
5   2010      218.056
6   2011      224.939
7   2012      229.594
8   2013      232.957
9   2014      236.736
10  2015      237.017
11  2016      240.007
12  2017      245.120
13  2018      251.107
14  2019      255.657
15  2020      258.811
16  2021      270.970
17  2022      292.655
18  2023      304.702

--------------------------------------------------------------------------------------------------------------------------------------
Inflate Marginal Social Cost (Damage) Factors using BLS CPI for All Urban Consumers (CPI-U)
--------------------------------------------------------------------------------------------------------------------------------------
DATAFRAME: Annual CPI-U f

### For Health-Related Emissions Adjust for different Value of a Statistical Life (VSL) values


print(f"""
Steps 3 and 4: Obtain BLS CPI-U Data and Inflate Current Social Cost of Carbon (SCC) to USD2023
      
EPA Median for 2% near term discount rate and most commonly mentioned value is 190 USD-2020 using the GIVE model.
Inflate 190 $USD-2020 Social Cost of Carbon to $USD-2023

SCC Values used in analysis are:
      LOW: ${round(EPA_SCC_USD2023_PER_MT_LOW, 2)} per mt CO2e
      BASE: ${round(EPA_SCC_USD2023_PER_MT_BASE, 2)} per mt CO2e
      HIGH: ${round(EPA_SCC_USD2023_PER_MT_HIGH, 2)} per mt CO2e
""")

In [8]:
print(f"""
===========================================================================================================================================================
LOOKUP MARGINAL SOCIAL COSTS FOR HEALTH-RELATED EMISSIONS
===========================================================================================================================================================
No Electricity Grid Uncertainty (Just Current Grid and Future Grid Projections)
      

""")


LOOKUP MARGINAL SOCIAL COSTS FOR HEALTH-RELATED EMISSIONS
No Electricity Grid Uncertainty (Just Current Grid and Future Grid Projections)
      




### Calculate End-use specific marginal damages
**I used the total emissions column for each of the end uses for the following reasons:**
- Most homes only have 1 of each end-use, so it is unlikely that the homes have a significant consumption values from different fuel types. Thus, the total consumption and total emissions column (sum of each dwelling units consumption by end-use for each fuel) is fine to use to calculate marginal damages (social cost)
- We can visualize the emissions in 2 by 2 grid (CO2, PM25, SO2, NOx) with each appliance's heating fuel in a different shape or color. 

### Baseline Marginal Damages: WHOLE-HOME

In [9]:
from cmu_tare_model.public_impact.calculate_lifetime_climate_impacts_sensitivity import *
from cmu_tare_model.public_impact.calculate_lifetime_health_impacts_sensitivity import *
print("""
-------------------------------------------------------------------------------------------------------
Step 5: Calculate End-use specific marginal damages
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Baseline Marginal Damages: WHOLE-HOME
-------------------------------------------------------------------------------------------------------
""")
# Make copies from scenario consumption to keep df smaller
print("\n", "Creating dataframe to store marginal damages calculations ...")
df_baseline_scenario_damages = df_euss_am_baseline_home.copy()

# calculate_climate_impacts(df, menu_mp, policy_scenario, df_baseline_damages=None)
df_euss_am_baseline_home, df_baseline_scenario_damages = calculate_lifetime_climate_impacts(df=df_euss_am_baseline_home,
                                                                                   menu_mp=menu_mp,
                                                                                   policy_scenario='No Inflation Reduction Act',
                                                                                   df_baseline_damages=df_baseline_scenario_damages,
                                                                                   )
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
PRE-IRA LONG RUN AND SHORT RUN MARGINAL EMISSIONS RATES (LRMER, SRMER) FROM CAMBIUM 2021 RELEASE
-------------------------------------------------------------------------------------------------------



Retrieved data for filename: cambium21_midCase_annual_gea.xlsx
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\projections\cambium21_midCase_annual_gea.xlsx

Loading dataframe ...
Creating lookup dictionary for LRMER and SRMER ...
-------------------------------------------------------------------------------------------------------


-------------------------------------------------------------------------------------------------------
IRA LONG RUN AND SHORT RUN MARGINAL EMISSIONS RATES (LRMER, SRMER) FROM CAMBIUM 2022 RELEASE
-------------------------------------------------------------------------------------------------------


Retrieved data for 

,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,county_fips,puma,county_and_puma,weather_file_city,Longitude,Latitude,building_type,income,federal_poverty_level,occupancy,tenure,vacancy_status,base_heating_fuel,heating_type,hvac_cooling_type,vintage,base_heating_efficiency,base_electricity_heating_consumption,base_fuelOil_heating_consumption,base_naturalGas_heating_consumption,base_propane_heating_consumption,base_waterHeating_fuel,waterHeating_type,base_electricity_waterHeating_consumption,base_fuelOil_waterHeating_consumption,base_naturalGas_waterHeating_consumption,base_propane_waterHeating_consumption,base_clothesDrying_fuel,clothesDrying_type,base_electricity_clothesDrying_consumption,base_naturalGas_clothesDrying_consumption,base_propane_clothesDrying_consumption,base_cooking_fuel,cooking_type,base_electricity_cooking_consumption,base_naturalGas_cooking_consumption,base_propane_cooking_consumption,baseline_heating_consumption,baseline_waterHeating_consumption,baseline_clothesDrying_consumption,baseline_cooking_consumption,baseline_2024_heating_consumption,baseline_2025_heating_consumption,baseline_2026_heating_consumption,baseline_2027_heating_consumption,baseline_2028_heating_consumption,baseline_2029_heating_consumption,baseline_2030_heating_consumption,baseline_2031_heating_consumption,baseline_2032_heating_consumption,baseline_2033_heating_consumption,baseline_2034_heating_consumption,baseline_2035_heating_consumption,baseline_2036_heating_consumption,baseline_2037_heating_consumption,baseline_2038_heating_consumption,baseline_2024_waterHeating_consumption,baseline_2025_waterHeating_consumption,baseline_2026_waterHeating_consumption,baseline_2027_waterHeating_consumption,baseline_2028_waterHeating_consumption,baseline_2029_waterHeating_consumption,baseline_2030_waterHeating_consumption,baseline_2031_waterHeating_consumption,baseline_2032_waterHeating_consumption,baseline_2033_waterHeating_consumption,baseline_2034_waterHeating_consumption,baseline_2035_waterHeating_consumption,baseline_2024_clothesDrying_consumption,baseline_2025_clothesDrying_consumption,baseline_2026_clothesDrying_consumption,baseline_2027_clothesDrying_consumption,baseline_2028_clothesDrying_consumption,baseline_2029_clothesDrying_consumption,baseline_2030_clothesDrying_consumption,baseline_2031_clothesDrying_consumption,baseline_2032_clothesDrying_consumption,baseline_2033_clothesDrying_consumption,baseline_2034_clothesDrying_consumption,baseline_2035_clothesDrying_consumption,baseline_2036_clothesDrying_consumption,baseline_2024_cooking_consumption,baseline_2025_cooking_consumption,baseline_2026_cooking_consumption,baseline_2027_cooking_consumption,baseline_2028_cooking_consumption,baseline_2029_cooking_consumption,baseline_2030_cooking_consumption,baseline_2031_cooking_consumption,baseline_2032_cooking_consumption,baseline_2033_cooking_consumption,baseline_2034_cooking_consumption,baseline_2035_cooking_consumption,baseline_2036_cooking_consumption,baseline_2037_cooking_consumption,baseline_2038_cooking_consumption,baseline_heating_lifetime_mt_co2e_lrmer,baseline_heating_lifetime_damages_climate_lrmer_lower,baseline_heating_lifetime_damages_climate_lrmer_central,baseline_heating_lifetime_damages_climate_lrmer_upper,baseline_heating_lifetime_mt_co2e_srmer,baseline_heating_lifetime_damages_climate_srmer_lower,baseline_heating_lifetime_damages_climate_srmer_central,baseline_heating_lifetime_damages_climate_srmer_upper,baseline_waterHeating_lifetime_mt_co2e_lrmer,baseline_waterHeating_lifetime_damages_climate_lrmer_lower,baseline_waterHeating_lifetime_damages_climate_lrmer_central,baseline_waterHeating_lifetime_damages_climate_lrmer_upper,baseline_waterHeating_lifetime_mt_co2e_srmer,baseline_waterHeating_lifetime_damages_climate_srmer_lower,baseline_waterHeating_lifetime_damages_climate_srmer_central,baseline_waterHeating_lifetime_damages_climate_srmer

In [10]:
# df_copy = df_euss_am_baseline_home.copy()

# cols_to_check = [
#     'county_fips',
#     'base_heating_fuel',
#     # 'base_clothesDrying_fuel',
#     # 'base_cooking_fuel',
#     'base_electricity_heating_consumption',
#     'base_fuelOil_heating_consumption',
#     'base_naturalGas_heating_consumption',
#     'base_propane_heating_consumption',
#     'baseline_heating_consumption',
#     'baseline_heating_lifetime_mt_co2e_lrmer',
#     'baseline_heating_lifetime_mt_co2e_srmer',
#     'base_waterHeating_fuel',
#     'base_electricity_waterHeating_consumption',
#     'base_fuelOil_waterHeating_consumption',
#     'base_naturalGas_waterHeating_consumption',
#     'base_propane_waterHeating_consumption',
#     'baseline_waterHeating_consumption',
#     'baseline_waterHeating_lifetime_mt_co2e_lrmer',
#     'baseline_waterHeating_lifetime_mt_co2e_srmer',
#     'base_clothesDrying_fuel',
#     'baseline_clothesDrying_consumption',
#     'baseline_clothesDrying_lifetime_mt_co2e_lrmer',
#     'baseline_clothesDrying_lifetime_mt_co2e_srmer',
#     'base_cooking_fuel',
#     'baseline_cooking_consumption',
#     'baseline_cooking_lifetime_mt_co2e_lrmer',
#     'baseline_cooking_lifetime_mt_co2e_srmer',
#     ]

# df_test = df_copy[cols_to_check]
# # print(df_test)
# df_test

In [11]:
df_euss_am_baseline_home

,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,county_fips,puma,county_and_puma,weather_file_city,Longitude,Latitude,building_type,income,federal_poverty_level,occupancy,tenure,vacancy_status,base_heating_fuel,heating_type,hvac_cooling_type,vintage,base_heating_efficiency,base_electricity_heating_consumption,base_fuelOil_heating_consumption,base_naturalGas_heating_consumption,base_propane_heating_consumption,base_waterHeating_fuel,waterHeating_type,base_electricity_waterHeating_consumption,base_fuelOil_waterHeating_consumption,base_naturalGas_waterHeating_consumption,base_propane_waterHeating_consumption,base_clothesDrying_fuel,clothesDrying_type,base_electricity_clothesDrying_consumption,base_naturalGas_clothesDrying_consumption,base_propane_clothesDrying_consumption,base_cooking_fuel,cooking_type,base_electricity_cooking_consumption,base_naturalGas_cooking_consumption,base_propane_cooking_consumption,baseline_heating_consumption,baseline_waterHeating_consumption,baseline_clothesDrying_consumption,baseline_cooking_consumption,baseline_2024_heating_consumption,baseline_2025_heating_consumption,baseline_2026_heating_consumption,baseline_2027_heating_consumption,baseline_2028_heating_consumption,baseline_2029_heating_consumption,baseline_2030_heating_consumption,baseline_2031_heating_consumption,baseline_2032_heating_consumption,baseline_2033_heating_consumption,baseline_2034_heating_consumption,baseline_2035_heating_consumption,baseline_2036_heating_consumption,baseline_2037_heating_consumption,baseline_2038_heating_consumption,baseline_2024_waterHeating_consumption,baseline_2025_waterHeating_consumption,baseline_2026_waterHeating_consumption,baseline_2027_waterHeating_consumption,baseline_2028_waterHeating_consumption,baseline_2029_waterHeating_consumption,baseline_2030_waterHeating_consumption,baseline_2031_waterHeating_consumption,baseline_2032_waterHeating_consumption,baseline_2033_waterHeating_consumption,baseline_2034_waterHeating_consumption,baseline_2035_waterHeating_consumption,baseline_2024_clothesDrying_consumption,baseline_2025_clothesDrying_consumption,baseline_2026_clothesDrying_consumption,baseline_2027_clothesDrying_consumption,baseline_2028_clothesDrying_consumption,baseline_2029_clothesDrying_consumption,baseline_2030_clothesDrying_consumption,baseline_2031_clothesDrying_consumption,baseline_2032_clothesDrying_consumption,baseline_2033_clothesDrying_consumption,baseline_2034_clothesDrying_consumption,baseline_2035_clothesDrying_consumption,baseline_2036_clothesDrying_consumption,baseline_2024_cooking_consumption,baseline_2025_cooking_consumption,baseline_2026_cooking_consumption,baseline_2027_cooking_consumption,baseline_2028_cooking_consumption,baseline_2029_cooking_consumption,baseline_2030_cooking_consumption,baseline_2031_cooking_consumption,baseline_2032_cooking_consumption,baseline_2033_cooking_consumption,baseline_2034_cooking_consumption,baseline_2035_cooking_consumption,baseline_2036_cooking_consumption,baseline_2037_cooking_consumption,baseline_2038_cooking_consumption,baseline_heating_lifetime_mt_co2e_lrmer,baseline_heating_lifetime_damages_climate_lrmer_lower,baseline_heating_lifetime_damages_climate_lrmer_central,baseline_heating_lifetime_damages_climate_lrmer_upper,baseline_heating_lifetime_mt_co2e_srmer,baseline_heating_lifetime_damages_climate_srmer_lower,baseline_heating_lifetime_damages_climate_srmer_central,baseline_heating_lifetime_damages_climate_srmer_upper,baseline_waterHeating_lifetime_mt_co2e_lrmer,baseline_waterHeating_lifetime_damages_climate_lrmer_lower,baseline_waterHeating_lifetime_damages_climate_lrmer_central,baseline_waterHeating_lifetime_damages_climate_lrmer_upper,baseline_waterHeating_lifetime_mt_co2e_srmer,baseline_waterHeating_lifetime_damages_climate_srmer_lower,baseline_waterHeating_lifetime_damages_climate_srmer_central,baseline_waterHeating_lifetime_damages_climate_srmer

In [12]:
# calculate_climate_impacts(df, menu_mp, policy_scenario, df_baseline_damages=None)
df_euss_am_baseline_home, df_baseline_scenario_damages = calculate_lifetime_health_impacts(df=df_euss_am_baseline_home,
                                                                                  menu_mp=menu_mp,
                                                                                  policy_scenario='No Inflation Reduction Act',
                                                                                  df_baseline_damages=df_baseline_scenario_damages,
                                                                                  )
df_euss_am_baseline_home

-- Scenario: Baseline -- 
              scenario_prefix: 'baseline_', cambium_scenario: 'MidCase', lookup_emissions_fossil_fuel: 'lookup_emissions_fossil_fuel', 
              lookup_emissions_electricity_climate: 'lookup_emissions_electricity_climate_preIRA', lookup_emissions_electricity_health: 'lookup_emissions_electricity_health'
              
Calculating Health Emissions and Damages from 2024 to 2039 for heating
Calculating Health Emissions and Damages from 2024 to 2036 for waterHeating
Calculating Health Emissions and Damages from 2024 to 2037 for clothesDrying
Calculating Health Emissions and Damages from 2024 to 2039 for cooking


,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,county_fips,puma,county_and_puma,weather_file_city,Longitude,Latitude,building_type,income,federal_poverty_level,occupancy,tenure,vacancy_status,base_heating_fuel,heating_type,hvac_cooling_type,vintage,base_heating_efficiency,base_electricity_heating_consumption,base_fuelOil_heating_consumption,base_naturalGas_heating_consumption,base_propane_heating_consumption,base_waterHeating_fuel,waterHeating_type,base_electricity_waterHeating_consumption,base_fuelOil_waterHeating_consumption,base_naturalGas_waterHeating_consumption,base_propane_waterHeating_consumption,base_clothesDrying_fuel,clothesDrying_type,base_electricity_clothesDrying_consumption,base_naturalGas_clothesDrying_consumption,base_propane_clothesDrying_consumption,base_cooking_fuel,cooking_type,base_electricity_cooking_consumption,base_naturalGas_cooking_consumption,base_propane_cooking_consumption,baseline_heating_consumption,baseline_waterHeating_consumption,baseline_clothesDrying_consumption,baseline_cooking_consumption,baseline_2024_heating_consumption,baseline_2025_heating_consumption,baseline_2026_heating_consumption,baseline_2027_heating_consumption,baseline_2028_heating_consumption,baseline_2029_heating_consumption,baseline_2030_heating_consumption,baseline_2031_heating_consumption,baseline_2032_heating_consumption,baseline_2033_heating_consumption,baseline_2034_heating_consumption,baseline_2035_heating_consumption,baseline_2036_heating_consumption,baseline_2037_heating_consumption,baseline_2038_heating_consumption,baseline_2024_waterHeating_consumption,baseline_2025_waterHeating_consumption,baseline_2026_waterHeating_consumption,baseline_2027_waterHeating_consumption,baseline_2028_waterHeating_consumption,baseline_2029_waterHeating_consumption,baseline_2030_waterHeating_consumption,baseline_2031_waterHeating_consumption,baseline_2032_waterHeating_consumption,baseline_2033_waterHeating_consumption,baseline_2034_waterHeating_consumption,baseline_2035_waterHeating_consumption,baseline_2024_clothesDrying_consumption,baseline_2025_clothesDrying_consumption,baseline_2026_clothesDrying_consumption,baseline_2027_clothesDrying_consumption,baseline_2028_clothesDrying_consumption,baseline_2029_clothesDrying_consumption,baseline_2030_clothesDrying_consumption,baseline_2031_clothesDrying_consumption,baseline_2032_clothesDrying_consumption,baseline_2033_clothesDrying_consumption,baseline_2034_clothesDrying_consumption,baseline_2035_clothesDrying_consumption,baseline_2036_clothesDrying_consumption,baseline_2024_cooking_consumption,baseline_2025_cooking_consumption,baseline_2026_cooking_consumption,baseline_2027_cooking_consumption,baseline_2028_cooking_consumption,baseline_2029_cooking_consumption,baseline_2030_cooking_consumption,baseline_2031_cooking_consumption,baseline_2032_cooking_consumption,baseline_2033_cooking_consumption,baseline_2034_cooking_consumption,baseline_2035_cooking_consumption,baseline_2036_cooking_consumption,baseline_2037_cooking_consumption,baseline_2038_cooking_consumption,baseline_heating_lifetime_mt_co2e_lrmer,baseline_heating_lifetime_damages_climate_lrmer_lower,baseline_heating_lifetime_damages_climate_lrmer_central,baseline_heating_lifetime_damages_climate_lrmer_upper,baseline_heating_lifetime_mt_co2e_srmer,baseline_heating_lifetime_damages_climate_srmer_lower,baseline_heating_lifetime_damages_climate_srmer_central,baseline_heating_lifetime_damages_climate_srmer_upper,baseline_waterHeating_lifetime_mt_co2e_lrmer,baseline_waterHeating_lifetime_damages_climate_lrmer_lower,baseline_waterHeating_lifetime_damages_climate_lrmer_central,baseline_waterHeating_lifetime_damages_climate_lrmer_upper,baseline_waterHeating_lifetime_mt_co2e_srmer,baseline_waterHeating_lifetime_damages_climate_srmer_lower,baseline_waterHeating_lifetime_damages_climate_srmer_central,baseline_waterHeating_lifetime_damages_climate_srmer

In [13]:
# from cmu_tare_model.public_impact.data_processing.create_lookup_health_impact_county import *

In [14]:
# print(df_health_rcm_ground_acs['model'].unique())

In [15]:
# # Count the rows with county_fips equal to "08014"
# num_rows = (df_euss_am_baseline_home['county_fips'] == '08014').sum()
# print(f"Number of rows with county_fips '08014': {num_rows}")

# # Drop rows with county_fips equal to "08014"
# df_euss_am_baseline_home = df_euss_am_baseline_home[df_euss_am_baseline_home['county_fips'] != '08014']
# print(f"Number of rows after dropping '08014': {len(df_euss_am_baseline_home)}")

In [16]:
# RCM_MODELS = ['AP2', 'EASIUR', 'InMAP']

# missing_any_models = []  # Will hold tuples of (county_key, [list_of_missing_models])

# for county_key, model_dict in lookup_health_fossil_fuel_acs.items():
#     existing_models = set(model_dict.keys())  # e.g. {'EASIUR', 'InMAP'} for that county
#     # Figure out which models from RCM_MODELS aren't present
#     missing = [m for m in RCM_MODELS if m not in existing_models]
#     if missing:
#         missing_any_models.append((county_key, missing))

# # Print a summary
# print("Counties missing at least one model:")
# for (county_key, missing_list) in missing_any_models:
#     print(f"  {county_key} is missing {missing_list}")


## Private Perspective: Annual Energy Costs

In [17]:
from cmu_tare_model.private_impact.process_fuel_price_data import *
print(f"""
===========================================================================================================================================================
PROCESS FUEL PRICE DATA AND PREPARE FOR PROJECTION: 
1. Load nominal fuel price data from EIA sources (2018-2022)
2. Convert base units (like dollars per gallon and cf) in nominal fuel price data to USD2023/kWh
3. Inflate all nominal fuel prices to $USD2023 (example: 2018 data in USD2018... multiply by cpi_ratio_2023_2018)
4. Map states to census division using map_location_to_census_division function (allows us to apply regional projection factors)
===========================================================================================================================================================
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity Price: https://www.eia.gov/electricity/state/archive/2018/
- EIA Natural Gas Prices: https://www.eia.gov/dnav/ng/ng_pri_sum_dcu_SPA_a.htm
- Propane and Fuel Oil: EIA March 2023 Short Term Energy Outlook
    - https://www.eia.gov/outlooks/steo/pdf/wf01.pdf
    - Table WF01: Average Consumer Prices and Expenditures for Heating Fuels During the Winter
    - US Average: 2018-2019 Data
===========================================================================================================================================================

The process_fuel_price_data.py file contains the following dataframes and lookup dictionaries:

DATAFRAME: Processed fuel price data (normalized in per kWh price, set up location columns to allow mapping for projection factors)
{df_fuelPrices_perkWh}
"""
)

Project root directory: c:\users\14128\research\cmu-tare-model
Retrieved data for filename: fuel_prices_nominal.csv
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\fuel_prices\fuel_prices_nominal.csv
Retrieved data for filename: aeo_projections_2022_2050.xlsx
Located at filepath: c:\users\14128\research\cmu-tare-model\cmu_tare_model\data\projections\aeo_projections_2022_2050.xlsx

PROCESS FUEL PRICE DATA AND PREPARE FOR PROJECTION: 
1. Load nominal fuel price data from EIA sources (2018-2022)
2. Convert base units (like dollars per gallon and cf) in nominal fuel price data to USD2023/kWh
3. Inflate all nominal fuel prices to $USD2023 (example: 2018 data in USD2018... multiply by cpi_ratio_2023_2018)
4. Map states to census division using map_location_to_census_division function (allows us to apply regional projection factors)
**Data Sources for Excel workbook containing state average Residential fuel cost for each fuel in 2018**
- EIA State Electricity P

In [18]:
print(f"""
===========================================================================================================================================================
PROJECT FUTURE FUEL PRICES: 
5. Load 2022-2050 regional projection factor data from AEO2023 (Normalized all fuel prices by the 2022 value for each region)
6. For each policy scenario, project future fuel prices using the project_future_prices function
7. For each policy scenario, create a lookup dictionary using the create_fuel_price_lookup function
===========================================================================================================================================================

The process_fuel_price_data.py file contains the following dataframes and lookup dictionaries:

DATAFRAME: Regional Fuel Price Projection Factors
{df_fuelPrices_projection_factors}

PRE-IRA SCENARIO
----------------------------------------------------------------------------------------------------------------------------------------------------------
DATAFRAME: Pre-IRA Projection Factors
{df_fuelPrices_perkWh_preIRA}

LOOKUP: Pre-IRA Projection Factors
{lookup_fuel_prices_preIRA}

IRA-REFERENCE SCENARIO
----------------------------------------------------------------------------------------------------------------------------------------------------------
DATAFRAME: IRA-Reference Projection Factors
{df_fuelPrices_perkWh_iraRef}

DATAFRAME: IRA-Reference Projection Factors
{lookup_fuel_prices_iraRef}
"""
)


PROJECT FUTURE FUEL PRICES: 
5. Load 2022-2050 regional projection factor data from AEO2023 (Normalized all fuel prices by the 2022 value for each region)
6. For each policy scenario, project future fuel prices using the project_future_prices function
7. For each policy scenario, create a lookup dictionary using the create_fuel_price_lookup function

The process_fuel_price_data.py file contains the following dataframes and lookup dictionaries:

DATAFRAME: Regional Fuel Price Projection Factors
       region    fuel_type                         policy_scenario  2022  \
0    National      propane                  AEO2022 Reference Case     1   
1    National      propane                  AEO2023 Reference Case     1   
2    National      propane         Low Zero-Carbon Technology Cost     1   
3    National      propane              No Inflation Reduction Act     1   
4    National      propane   Low Uptake of Inflation Reduction Act     1   
..        ...          ...                  

### Step 2: Calculate Annual Operating (Fuel) Costs

### Baseline Fuel Cost: WHOLE-HOME

In [19]:
from cmu_tare_model.private_impact.calculate_fuel_costs import *

print("""
-------------------------------------------------------------------------------------------------------
Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 
-------------------------------------------------------------------------------------------------------
""")
# calculate_annual_fuelCost(df, menu_mp, policy_scenario, drop_fuel_cost_columns)
df_euss_am_baseline_home = calculate_annual_fuelCost(df=df_euss_am_baseline_home,
                                                     menu_mp=menu_mp,
                                                     policy_scenario='No Inflation Reduction Act',
                                                     drop_fuel_cost_columns=False
                                                     )
df_euss_am_baseline_home


-------------------------------------------------------------------------------------------------------
Step 2: Calculate Annual Operating (Fuel) Costs
-------------------------------------------------------------------------------------------------------
- Create a mapping dictionary for fuel types
- Create new merge columns to ensure a proper match.
- Merge df_copy with df_fuel_prices to get fuel prices for electricity, natural gas, propane, and fuel oil
- Calculate the per kWh fuel costs for each fuel type and region
- Calculate the baseline fuel cost 
-------------------------------------------------------------------------------------------------------

Calculating BASELINE (no retrofit) fuel costs from 2024 to 2039 for heating
Calculating BASELINE (no retrofit) fuel costs from 2024 to 2036 for waterHeating
Calculating BASELINE (no retrofit) fuel costs from 2024 to 2037 for clothesDrying
Calculating BASELINE (no retrofit) fuel costs from 2024 to 2039 for cooking


,square_footage,census_region,census_division,census_division_recs,building_america_climate_zone,reeds_balancing_area,gea_region,state,city,county,county_fips,puma,county_and_puma,weather_file_city,Longitude,Latitude,building_type,income,federal_poverty_level,occupancy,tenure,vacancy_status,base_heating_fuel,heating_type,hvac_cooling_type,vintage,base_heating_efficiency,base_electricity_heating_consumption,base_fuelOil_heating_consumption,base_naturalGas_heating_consumption,base_propane_heating_consumption,base_waterHeating_fuel,waterHeating_type,base_electricity_waterHeating_consumption,base_fuelOil_waterHeating_consumption,base_naturalGas_waterHeating_consumption,base_propane_waterHeating_consumption,base_clothesDrying_fuel,clothesDrying_type,base_electricity_clothesDrying_consumption,base_naturalGas_clothesDrying_consumption,base_propane_clothesDrying_consumption,base_cooking_fuel,cooking_type,base_electricity_cooking_consumption,base_naturalGas_cooking_consumption,base_propane_cooking_consumption,baseline_heating_consumption,baseline_waterHeating_consumption,baseline_clothesDrying_consumption,baseline_cooking_consumption,baseline_2024_heating_consumption,baseline_2025_heating_consumption,baseline_2026_heating_consumption,baseline_2027_heating_consumption,baseline_2028_heating_consumption,baseline_2029_heating_consumption,baseline_2030_heating_consumption,baseline_2031_heating_consumption,baseline_2032_heating_consumption,baseline_2033_heating_consumption,baseline_2034_heating_consumption,baseline_2035_heating_consumption,baseline_2036_heating_consumption,baseline_2037_heating_consumption,baseline_2038_heating_consumption,baseline_2024_waterHeating_consumption,baseline_2025_waterHeating_consumption,baseline_2026_waterHeating_consumption,baseline_2027_waterHeating_consumption,baseline_2028_waterHeating_consumption,baseline_2029_waterHeating_consumption,baseline_2030_waterHeating_consumption,baseline_2031_waterHeating_consumption,baseline_2032_waterHeating_consumption,baseline_2033_waterHeating_consumption,baseline_2034_waterHeating_consumption,baseline_2035_waterHeating_consumption,baseline_2024_clothesDrying_consumption,baseline_2025_clothesDrying_consumption,baseline_2026_clothesDrying_consumption,baseline_2027_clothesDrying_consumption,baseline_2028_clothesDrying_consumption,baseline_2029_clothesDrying_consumption,baseline_2030_clothesDrying_consumption,baseline_2031_clothesDrying_consumption,baseline_2032_clothesDrying_consumption,baseline_2033_clothesDrying_consumption,baseline_2034_clothesDrying_consumption,baseline_2035_clothesDrying_consumption,baseline_2036_clothesDrying_consumption,baseline_2024_cooking_consumption,baseline_2025_cooking_consumption,baseline_2026_cooking_consumption,baseline_2027_cooking_consumption,baseline_2028_cooking_consumption,baseline_2029_cooking_consumption,baseline_2030_cooking_consumption,baseline_2031_cooking_consumption,baseline_2032_cooking_consumption,baseline_2033_cooking_consumption,baseline_2034_cooking_consumption,baseline_2035_cooking_consumption,baseline_2036_cooking_consumption,baseline_2037_cooking_consumption,baseline_2038_cooking_consumption,baseline_heating_lifetime_mt_co2e_lrmer,baseline_heating_lifetime_damages_climate_lrmer_lower,baseline_heating_lifetime_damages_climate_lrmer_central,baseline_heating_lifetime_damages_climate_lrmer_upper,baseline_heating_lifetime_mt_co2e_srmer,baseline_heating_lifetime_damages_climate_srmer_lower,baseline_heating_lifetime_damages_climate_srmer_central,baseline_heating_lifetime_damages_climate_srmer_upper,baseline_waterHeating_lifetime_mt_co2e_lrmer,baseline_waterHeating_lifetime_damages_climate_lrmer_lower,baseline_waterHeating_lifetime_damages_climate_lrmer_central,baseline_waterHeating_lifetime_damages_climate_lrmer_upper,baseline_waterHeating_lifetime_mt_co2e_srmer,baseline_waterHeating_lifetime_damages_climate_srmer_lower,baseline_waterHeating_lifetime_damages_climate_srmer_central,baseline_waterHeating_lifetime_damages_climate_srmer

# Model Runtime

In [20]:
# Get the current datetime again
end_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Calculate the elapsed time
elapsed_time = datetime.strptime(end_time, "%Y-%m-%d_%H-%M-%S") - datetime.strptime(start_time, "%Y-%m-%d_%H-%M-%S")

# Format the elapsed time
elapsed_seconds = elapsed_time.total_seconds()
elapsed_minutes = int(elapsed_seconds // 60)
elapsed_seconds = int(elapsed_seconds % 60)

# Print the elapsed time
print(f"The code took {elapsed_minutes} minutes and {elapsed_seconds} seconds to execute.")

The code took 11 minutes and 6 seconds to execute.
